In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, MaxAbsScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing  # Dataset exemple
import numpy as np
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import random
from tqdm import tqdm
import pandas as pd
from prophet import Prophet
from tqdm import tqdm
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import random
from tqdm import tqdm


In [14]:
df = pd.read_csv("data.csv")

In [15]:
df.head()

,Domaine,Zone,Élément,Produit,Année,Valeur
0,Cultures et produits animaux,Maroc,Production,Ail frais,1961,2000.0
1,Cultures et produits animaux,Maroc,Production,Ail frais,1962,2000.0
2,Cultures et produits animaux,Maroc,Production,Ail frais,1963,3000.0
3,Cultures et produits animaux,Maroc,Production,Ail frais,1964,2000.0
4,Cultures et produits animaux,Maroc,Production,Ail frais,1965,1000.0


In [16]:
df.describe()

,Année,Valeur
count,1070.000000,1042.000000
mean,1991.971028,55071.899827
std,18.176510,110423.877949
min,1961.000000,0.000000
25%,1976.000000,2500.000000
50%,1992.000000,17375.000000
75%,2008.000000,50730.125000
max,2023.000000,922820.040000


In [17]:
df = df[['Année', 'Produit', 'Valeur']]

In [18]:
df.dropna(inplace=True)

In [19]:
df['Année'] = df['Année'].astype(int)
df['Année'] = pd.to_datetime(df['Année'], format='%Y')

In [20]:
Q1 = df["Valeur"].quantile(0.25)
Q3 = df["Valeur"].quantile(0.75)

IQR = Q3 - Q1

df = df[(df["Valeur"] >= (Q1 -1.5 * IQR)) & (df["Valeur"] <= (Q1 + 1.5 * IQR))]

In [21]:
df = df.rename(columns={'Année': 'ds','Produit' : 'product', 'Valeur': 'y'}).dropna()

In [22]:
df.head(4)

,ds,product,y
0,1961-01-01,Ail frais,2000.0
1,1962-01-01,Ail frais,2000.0
2,1963-01-01,Ail frais,3000.0
3,1964-01-01,Ail frais,2000.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 866 entries, 0 to 1069
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ds       866 non-null    datetime64[ns]
 1   product  866 non-null    object        
 2   y        866 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 27.1+ KB


# Entrainement :

In [28]:
def is_better(new_r2, new_mae, new_rmse, best_r2, best_mae, best_rmse):
    return (new_r2 > best_r2) or (new_rmse < best_rmse) or (new_mae < best_mae)


produits = df["product"].unique()
nb_essais = 100  

lignes_best = []   
futures = []       

for produit in produits:
    data = df.loc[df["product"] == produit, ["ds", "y"]].copy()
    if len(data) < 5:
        continue

    meilleur_r2   = -np.inf
    meilleur_mae  = np.inf
    meilleur_rmse = np.inf
    meilleur_params = None

    for _ in range(nb_essais):
            params = {
                "changepoint_prior_scale": random.uniform(0.001, 0.5),
                "seasonality_prior_scale": random.uniform(0.01, 20),
                "holidays_prior_scale": random.uniform(0.01, 20),
                "seasonality_mode": random.choice(["additive", "multiplicative"]),
                "changepoint_range": random.uniform(0.8, 0.95),
            }

            model = Prophet(**params)
            model.fit(data)

            forecast_hist = model.predict(data[["ds"]])
            y_true = data["y"].values
            y_pred = forecast_hist["yhat"].values

            r2   = r2_score(y_true, y_pred)
            mae  = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))

            if is_better(r2, mae, rmse, meilleur_r2, meilleur_mae, meilleur_rmse):
                meilleur_r2, meilleur_mae, meilleur_rmse = r2, mae, rmse
                meilleur_params = params


    if meilleur_params is not None:
        
            best_model = Prophet(**meilleur_params)
            best_model.fit(data)

            future_dates = pd.date_range(start="2023-01-01", end="2040-01-01", freq="YS")
            forecast_future = best_model.predict(pd.DataFrame({"ds": future_dates}))

            prev = forecast_future[["ds", "yhat"]].copy()
            prev["product"] = produit
            futures.append(prev)

            lignes_best.append({
                "product": produit,
                "r2_score": meilleur_r2,
                "mae": meilleur_mae,
                "rmse": meilleur_rmse,
                **meilleur_params
            })


df_final = pd.DataFrame(lignes_best)
if futures:
    df_futures = pd.concat(futures, ignore_index=True)
    df_futures.rename(columns={"ds": "date", "yhat": "prediction_en_tonnes"}, inplace=True)
    df_futures = df_futures[["date", "product", "prediction_en_tonnes"]]
else:
    df_futures = pd.DataFrame(columns=["date", "product", "prediction_en_tonnes"])

print("df_final : meilleurs hyperparamètres")
print("df_futures : prévisions 2040 par produit")


20:21:14 - cmdstanpy - INFO - Chain [1] start processing
20:21:16 - cmdstanpy - INFO - Chain [1] done processing
20:21:16 - cmdstanpy - INFO - Chain [1] start processing
20:21:16 - cmdstanpy - INFO - Chain [1] done processing
20:21:17 - cmdstanpy - INFO - Chain [1] start processing
20:21:18 - cmdstanpy - INFO - Chain [1] done processing
20:21:18 - cmdstanpy - INFO - Chain [1] start processing
20:21:19 - cmdstanpy - INFO - Chain [1] done processing
20:21:19 - cmdstanpy - INFO - Chain [1] start processing
20:21:20 - cmdstanpy - INFO - Chain [1] done processing
20:21:20 - cmdstanpy - INFO - Chain [1] start processing
20:21:20 - cmdstanpy - INFO - Chain [1] done processing
20:21:21 - cmdstanpy - INFO - Chain [1] start processing
20:21:21 - cmdstanpy - INFO - Chain [1] done processing
20:21:21 - cmdstanpy - INFO - Chain [1] start processing
20:21:23 - cmdstanpy - INFO - Chain [1] done processing
20:21:23 - cmdstanpy - INFO - Chain [1] start processing
20:21:24 - cmdstanpy - INFO - Chain [1]

✅ Terminé :
 - df_final : meilleurs hyperparamètres + métriques par produit
 - df_futures : prévisions 2023→2040 par produit


In [29]:
df_final

,product,r2_score,mae,rmse,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,changepoint_range
0,Ail frais,0.887259,1185.866686,2175.779308,0.334776,13.697593,13.426043,multiplicative,0.828042
1,Artichauts,0.943448,2250.525125,3076.766618,0.390952,16.129063,15.650051,multiplicative,0.881326
2,Aubergines,0.960428,2198.606063,4016.086773,0.495820,10.087295,9.831666,multiplicative,0.918755
3,Avocats,0.988681,915.099234,1773.000770,0.467033,4.924286,19.721060,multiplicative,0.913613
4,Carottes et navets,1.000000,0.578324,0.625832,0.245746,18.101986,14.567736,multiplicative,0.940832
5,Cerises,0.917847,823.709771,1562.467735,0.442154,6.339040,10.824805,multiplicative,0.900478
6,Champignons et truffes,0.998707,14.192286,25.250909,0.461550,5.760542,7.091561,multiplicative,0.808027
7,Citrons et limes,0.955689,1881.392239,2432.287127,0.375525,7.999900,12.077843,multiplicative,0.894586
8,Coings,0.972892,1795.129175,2982.211118,0.386817,16.532275,19.396317,multiplicative,0.933726
9,"Concombres, cornichons",0.969865,2232.641683,3769.419494,0.227604,16.474997,15.023392,multiplicative,0.886743


In [28]:
df_futures

,date,product,prediction_en_tonnes
0,2023-01-01,Ail frais,17841.346915
1,2024-01-01,Ail frais,18908.956959
2,2025-01-01,Ail frais,19574.760638
3,2026-01-01,Ail frais,20669.609376
4,2027-01-01,Ail frais,21761.142675
...,...,...,...
301,2036-01-01,Vin,49422.608747
302,2037-01-01,Vin,50899.089051
303,2038-01-01,Vin,51307.911790
304,2039-01-01,Vin,51673.991878


In [29]:
df_futures.to_csv("prevision_2040.csv", index=False)